In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [24]:
no_pages = 2
search = "dan+brown"

def get_data(pageNo, searchTitle):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    #https://www.travessa.com.br/Busca.aspx?d=1&bt=dan%20brown&cta=00&codtipoartigoexplosao=1&pag=2
    r = requests.get('https://www.travessa.com.br/Busca.aspx?d=1&bt='+str(searchTitle)+'&cta=00&codtipoartigoexplosao=1&pag='+ str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []

    #table = soup.find('table', attrs={'id':'gridBusca'})
    #print(table)
    #rows = table.find_all('tr')
    
    for d in soup.findAll('table', attrs={'style':'border-bottom-style:solid;border-bottom-width:1px;border-bottom-color:darkgrey;width:100%;'}):
        #print(d)
        name = d.find('a', attrs={'class':'buscaResultadoTitulo'})
        #n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('span', attrs={'class':'buscaResultadoAutorProdutor'})
        #rating = d.find('span', attrs={'class':'4,1 de 5 estrelas'})
        #r = rating.find_all('span', aria-label=True)
        
        price = d.find('span', attrs={'class':'buscaResultadoPrecoPor'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(name.text)
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        #if rating is not None:
            #print(rating.text)
        #    all1.append(rating.text)
        #else:
        #    all1.append('-1')

        #if users_rated is not None:
            #print(price.text)
        #    all1.append(users_rated.text)
        #else:
        #    all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [25]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i, search))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Titulo','Autor', 'Preco'])
df.to_csv('travessa_products.csv', index=False, encoding='utf-8')

In [26]:
df = pd.read_csv("travessa_products.csv")
df.shape

(20, 3)

In [27]:
df.head(61)

,Titulo,Autor,Preco
0,O SIMBOLO PERDIDO - 2ªED.(2021),Dan Brown,"Por: R$ 33,92"
1,SINFONIA DOS ANIMAIS - 1ªED.(2020),Dan Brown,"Por: R$ 42,42"
2,O CODIGO DA VINCI - 1ªED.(2021),Dan Brown,"Por: R$ 33,92"
3,ORIGEM - 1ªED.(2020),Dan Brown,"Por: R$ 31,37"
4,ORIGEM - 1ªED.(2017),Dan Brown,"Por: R$ 46,67"
5,PONTO DE IMPACTO - 1ªED.(2005),Dan Brown,"Por: R$ 46,67"
6,FORTALEZA DIGITAL - 1ªED.(2015),Dan Brown,"Por: R$ 42,42"
7,INFERNO - 1ªED.(2013),Dan Brown,"R$ 54,90"
8,ANJOS E DEMONIOS: A PRIMEIRA AVENTURA DE ROBER...,Dan Brown,"R$ 54,90"
9,GUIA PARA A CHAVE DE SALOMAO DE DAN BROWN - 1ª...,Greg Taylor,"R$ 59,90"
